<a href="https://colab.research.google.com/github/Nigouz/3204-CourseWork2/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import re
from sklearn import preprocessing

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
# df = pd.read_json('/content/drive/MyDrive/3204 CourseWork 2/Gp15_dataset/raw_logs/filebeat_raw_Gp15_all.json', lines=True)
# df = pd.read_csv("https://98f0-175-156-193-95.ngrok.io/truncated.csv")
full_df = pd.read_csv(r"shuffled-noIPV6.csv")
# df = pd.read_csv("truncated_training.csv")

print(f"[*] Shape of dataset: {full_df.shape}")
print(full_df.tail())
print("\n")

[*] Shape of dataset: (98274, 15)
       Unnamed: 0    @timestamp           src_ip src_port         dst_ip  \
98269       98270  1.666692e+09      10.32.4.176    49012  10.132.100.53   
98270       98271  1.666580e+09  192.168.207.142        0              0   
98271       98272  1.666692e+09    100.64.19.103    44618   100.64.19.22   
98272       98273  1.666580e+09  192.168.207.142        0              0   
98273       98274  1.666692e+09    100.64.19.103    44618   100.64.19.22   

      dst_port Protocol http_request_method http_request_referrer  \
98269     1192      TCP                   0                     0   
98270        0    HTTPS                 GET                     0   
98271     8443      TCP                   0                     0   
98272        0    HTTPS                 GET                     0   
98273     8443      TCP                   0                     0   

      http_response_code                                           url_path  \
98269          

In [ ]:
full_df["src_port"]=  full_df["src_port"].replace(regex=",", value= "")
full_df["src_port"] = full_df["src_port"].replace(regex=" ", value="")

"""from google.colab import files
df.to_csv('output.csv', encoding = 'utf-8-sig') 
files.download('output.csv')"""

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn import metrics
scores = {}
scores_list= []

def scoring_metrics(y_test, y_pred, model):
    accuracy = metrics.accuracy_score(y_test, y_pred)
    scores[model] = accuracy
    scores_list.append(accuracy)
    confusion_matrix = metrics.confusion_matrix(y_true=[True, True], y_pred=[True, True], labels=[True, False])

    print(f"Confusion Matrix: {confusion_matrix}")

    print(f"\n[*] Model: {model}")
    print("[*] [green bold]True Positive:[/green bold] {}".format(confusion_matrix[0][0]))
    print("[*] [red bold]False Positive:[/red bold] {}".format(confusion_matrix[1][0]))
    print("[*] [red bold]False Negative:[/red bold] {}".format(confusion_matrix[0][1]))
    print("[*] [green bold]True Negative:[/green bold] {}".format(confusion_matrix[1][1]))
    print("[*] Precision: {:.3f}%".format(metrics.precision_score(y_test, y_pred)))
    print("[*] Recall: {:.3f}%".format(metrics.recall_score(y_test, y_pred)))

    print("[*] Accuracy: {:.3f}%".format(accuracy))
    print("[*] F1_score: {:.3f}%".format(metrics.f1_score(y_test, y_pred)))


# KNN

In [ ]:
print("Doing")
from sklearn.preprocessing import OneHotEncoder
df.replace(to_replace=["None"], value=np.nan, inplace=True)
clean_df = df.fillna(str(0)) #uhm i dont knowhow to explain this, but please lmk another alternative to fix this cause :") try to run it without str and you will know what i mean"
clean_x = clean_df.iloc[:, :14].values
clean_y = clean_df["category"].values
features = df.columns.values[:-1]

for label in clean_df.columns:
    for index, rows in clean_df.iterrows():
        new_ip = ""
        ip = str(rows[label])
        if re.search("\d+\.\d+\.\d+\.\d+", ip):
            octets = ip.split(".")
            for octet in octets:
                octet = octet.rjust(3,"0")
                new_ip += octet
            clean_df[label][index] = new_ip

clean_df["http_response_code"] = clean_df["http_response_code"].replace('HTTP/1.1"', value="0")
print("Done")

Doing


C:\Users\kahow\anaconda3\lib\site-packages\pandas\core\frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-10-35ad58ccf036>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[label][index] = new_ip


Done


In [ ]:
from sklearn.compose import make_column_transformer
clean_x = clean_df.iloc[:, :14] #honestly don't know if this is correct lmaoooo
#this is to iterate through the columns and convert the strings to float (passthrough means for those columns we didnt specify, leave it as it is)
column_trans = make_column_transformer((OneHotEncoder(sparse=False), ['Protocol', 'http_request_method', 'http_request_referrer', 'url_path', 'user_agent_original', 'sql_method', 'sql_query']),remainder='passthrough')
test = column_trans.fit_transform(clean_x) #this is technically our cl

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
x_train, x_test, y_train, y_test = train_test_split(test, clean_y, test_size=0.3)

In [ ]:
k = 5
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(x_train, y_train)
print(f"Saving trained model to test-knn{k}.sav")
pickle.dump(knn, open(f"test-knn{k}.sav", "wb"))
print("Model Saved")

# print("Loading knn from pickle")
# knn = pickle.load(open("knn5.sav", "rb"))
# print("KNN Successfully loaded")

y_pred = knn.predict(x_test)
scoring_metrics(y_test, y_pred, f"knn {k}")

ValueError: could not convert string to float: '46,898'